In [ ]:
import sqlite3

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    text

)

from sqlalchemy.orm import sessionmaker

import pandas as pd     

import numpy as np

In [ ]:
df = pd.read_json('catalog.json').T

In [ ]:

print(df.columns)

In [ ]:
#course offered contains values outside of seasons

unique_quarters = df['course_offered'].unique()

print([quarter for quarter in unique_quarters if quarter is not None and len(quarter) < 9])


In [ ]:
#filter out values that are not seasons
seasons = ['S.', 'A.', 'W.', 'Sp.', 'AS.']

df = df[df['course_offered'].isin(seasons)]

print(len(df['course_offered'].isin(seasons)))


In [ ]:
seasons = ['S.', 'A.', 'W.', 'Sp.']
print(df['course_offered'].isin(seasons).sum())

In [ ]:
#confirm class id is unique
print(len(df))
print(df['course_id'].nunique())

In [ ]:
#set up gpa 

gpa_data = df['gpa_distro'].apply(pd.Series)
gpa_data = gpa_data.applymap(lambda x: x.get('count', 0) if isinstance(x, (dict,)) else None)
print(gpa_data.columns)
print(gpa_data.head())

In [ ]:
#calculating mean

gpa_values = np.array([i / 10 for i in range(41)], dtype = float)

total_counts = gpa_data[[i for i in range(41)]].sum(axis=1)

mean_gpa = (gpa_data * gpa_values).sum(axis=1) / total_counts
mean_gpa.name = 'mean_gpa'



In [ ]:
df_mean_gpa = pd.concat([df, mean_gpa], axis=1)
print(df_mean_gpa.columns)
print(type(mean_gpa))

In [ ]:
df_dropped = df_mean_gpa.drop(columns=['gpa_distro', 'concurrent_courses', 'prereq_graph', 'coi_data', 'is_gateway', 'is_bottleneck', 'prereq_string'], inplace = False)
df_final = df_dropped.rename(columns = {'course_offered': 'quarters_offered'})


In [ ]:
print(df_final.columns)
print(df_final.dtypes)
print(df_final['course_id'].head())
print(type(df_final['course_credits']))
print(df_final.info())

In [ ]:
print(df_final.loc[df_final['department_abbrev'].str.contains('CS')]['department_abbrev'].unique())
print(df_final['course_credits'].head())

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('courses_temp.db')
cursor = conn.cursor()

# Define table name
table_name = 'class_data'  # Adjust this to match your actual table name


# Insert DataFrame into the SQLite table
df_final.to_sql(table_name, conn, index=False)

In [ ]:
departments = df_final['department_abbrev'].unique()
print(departments)

print('CSE' in departments)

In [ ]:
print(df_final['course_id'].head())